<a href="https://colab.research.google.com/github/21cs076/Real-Time-Landslide-Detection-and-Buried-Vehicle-Identification-using-YOLOv10/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.8/914.8 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 19.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="4MptKsSsONT0fnQsIDc2")
project = rf.workspace("deslizamentos").project("landslide-detection-augmentation")
version = project.version(12)
dataset = version.download("yolov11")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 12.8 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.11.0.86
    Uninstalling opencv-python-headless-4.11.0.86:
      Successfully uninstalled opencv-python-headless-4.11.0.86
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Landslide-Detection-Augmentation-12 in yolov11:: 100%|██████████| 2738/2738 [00:01<00:00, 1387.69it/s]


In [3]:
import albumentations as A
import cv2
import os
import numpy as np

# Define augmentation pipeline
transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=40, p=0.5),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='yolo', label_fields=['class_labels'], min_visibility=0.2))

def read_label(label_path):
    labels = []
    with open(label_path, 'r') as file:
        for line in file.readlines():
            if line.strip():
                class_id, x, y, w, h = map(float, line.strip().split())
                labels.append([class_id, x, y, w, h])
    return np.array(labels)

def save_label(label_path, labels):
    with open(label_path, 'w') as file:
        for label in labels:
            file.write(' '.join(map(str, label)) + '\n')

def augment_and_save(image_path, label_path, save_image_path, save_label_path, transform, num_augmented):
    image = cv2.imread(image_path)
    labels = read_label(label_path)

    if labels.size == 0:  # Skip augmentation if no labels are found
        return

    class_labels = labels[:, 0]
    bboxes = labels[:, 1:]

    for i in range(num_augmented):
        transformed = transform(image=image, bboxes=bboxes, class_labels=class_labels)
        augmented_image = transformed['image']
        augmented_bboxes = transformed['bboxes']

        if len(augmented_bboxes) == 0:  # Skip if no bounding boxes remain after augmentation
            continue

        augmented_labels = np.hstack((np.array(transformed['class_labels'])[:, np.newaxis], augmented_bboxes))

        augmented_image_path = os.path.join(save_image_path, f"aug_{i}_{os.path.basename(image_path)}")
        augmented_label_path = os.path.join(save_label_path, f"aug_{i}_{os.path.basename(label_path)}")

        cv2.imwrite(augmented_image_path, augmented_image)
        save_label(augmented_label_path, augmented_labels)

# Paths to your dataset images and labels
dataset_image_path = f"{dataset.location}/train/images"
dataset_label_path = f"{dataset.location}/train/labels"
augmented_image_path = f"{dataset.location}/aug/images"
augmented_label_path = f"{dataset.location}/aug/labels"

os.makedirs(augmented_image_path, exist_ok=True)
os.makedirs(augmented_label_path, exist_ok=True)

# Augment each image and its corresponding label in the dataset
for image_name in os.listdir(dataset_image_path):
    image_path = os.path.join(dataset_image_path, image_name)
    label_path = os.path.join(dataset_label_path, image_name.replace('.jpg', '.txt'))
    augment_and_save(image_path, label_path, augmented_image_path, augmented_label_path, transform, num_augmented=5)

In [4]:
import shutil
import os

# Ensure the augmented directories exist
os.makedirs(augmented_image_path, exist_ok=True)
os.makedirs(augmented_label_path, exist_ok=True)

# Copy original images to the augmented images directory
for image_name in os.listdir(dataset_image_path):
    src_image_path = os.path.join(dataset_image_path, image_name)
    dst_image_path = os.path.join(augmented_image_path, image_name)
    shutil.copy(src_image_path, dst_image_path)

# Copy original labels to the augmented labels directory
for label_name in os.listdir(dataset_label_path):
    src_label_path = os.path.join(dataset_label_path, label_name)
    dst_label_path = os.path.join(augmented_label_path, label_name)
    shutil.copy(src_label_path, dst_label_path)


import yaml
data = {
    'train': f"{dataset.location}/aug/images",
    'val': f"{dataset.location}/valid/images",
    'test': f"{dataset.location}/test/images",
    'nc': 2,
    'names': ['Fissure', 'Landslide']
}
with open(f"{dataset.location}/data.yaml", 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

In [5]:
from ultralytics import YOLO
from google.colab import files

model = YOLO("yolo11s.pt")
model.train(
    data = f"{dataset.location}/data.yaml",  # Path to your data.yaml file
    epochs=80,  # Number of epochs (adjust based on your needs)
    batch=16,   # Batch size (adjust based on GPU memory), (16), 32, 64, 128
    imgsz=640,  # Image size for training ((640 is a good choice), 736)
    project = f"{dataset.location}",  # Directory to save training results
    name="runs",  # Name for the training run
    exist_ok=True  # Overwrite existing results if any
)
files.download(f"{dataset.location}/runs/weights/best.pt")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 18.4M/18.4M [00:00<00:00, 124MB/s]


New https://pypi.org/project/ultralytics/8.3.73 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/Landslide-Detection-Augmentation-12/data.yaml, epochs=80, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/Landslide-Detection-Augmentation-12, name=runs, exist_ok=True, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, clas

100%|██████████| 755k/755k [00:00<00:00, 22.7MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  3                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  4                  -1  1    103360  ultralytics.nn.modules.block.C3k2            [128, 256, 1, False, 0.25]    
  5                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  6                  -1  1    346112  ultralytics.nn.modules.block.C3k2            [256, 256, 1, True]           
  7                  -1  1   1180672  ultralytics

...

                   all        115        162      0.626      0.482      0.515      0.242

80 epochs completed in 3.822 hours.
Optimizer stripped from /content/Landslide-Detection-Augmentation-12/runs/weights/last.pt, 19.2MB
Optimizer stripped from /content/Landslide-Detection-Augmentation-12/runs/weights/best.pt, 19.2MB

Validating /content/Landslide-Detection-Augmentation-12/runs/weights/best.pt...
Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 238 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.86it/s]


                   all        115        162      0.583      0.501      0.527      0.248
               Fissure         29         60      0.574      0.404      0.461      0.187
             Landslide         61        102      0.592      0.598      0.593      0.309
Speed: 0.1ms preprocess, 2.8ms inference, 0.0ms loss, 5.7ms postprocess per image
Results saved to /content/Landslide-Detection-Augmentation-12/runs


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [6]:
from ultralytics import YOLO
model = YOLO(f"{dataset.location}/runs/weights/best.pt")
#model = YOLO("/content/drive/MyDrive/models/lbest70m.pt")

metrics = model.val(data=f"{dataset.location}/data.yaml", split='test')
print(f"mAP50: {metrics.box.map50}") # mAP refers to the Average Precision at a single IoU threshold, often 50%.
print(f"mAP75: {metrics.box.map75}") # mAP at IoU threshold of 75% (mAP75)

Ultralytics 8.3.72 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 238 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning /content/Landslide-Detection-Augmentation-12/test/labels... 60 images, 21 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<00:00, 346.61it/s]

val: New cache created: /content/Landslide-Detection-Augmentation-12/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:02<00:00,  1.73it/s]


                   all         60         97      0.603       0.55      0.542      0.269
               Fissure         19         30       0.53      0.533      0.476      0.248
             Landslide         30         67      0.676      0.567      0.607      0.289
Speed: 0.3ms preprocess, 18.1ms inference, 0.0ms loss, 4.0ms postprocess per image
Results saved to runs/detect/val
mAP50: 0.5415869063090599
mAP75: 0.25017783714653685
